## 빅데이터 실습

### 다나와 무선청소기 상품분석

#### 데이터 수집

##### 셀레니움 사용
- 무선청소기 검색에 사용할 URL
    https://search.danawa.com/dsearch.php?query=무선청소기&originalQuery=무선청소기&previousKeyword=무선청소기&checkedInfo=N&volumeType=allvs&page=1&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A
    - URL 최소화 링크
    https://search.danawa.com/dsearch.php?query=무선청소기&page=2&limit=120&sort=saveDESC&list=list
    한페이지가 다 로드될때 5초정도의 시간이 필요

In [2]:
## 필수 라이브러리 사용등록
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse    # url encode
from tqdm import tqdm
import time

In [3]:
## 웹드라이버로 크롬 오픈
driver = webdriver.Chrome()
url = r'https://search.danawa.com/dsearch.php?query=무선청소기&page=1&limit=120&sort=saveDESC&list=list'
driver.get(url)
time.sleep(0.5)

In [4]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [5]:
soup.select('li.prod_item')

[<li class="prod_item" id="productItem19965230">
 <input id="productItem_categoryInfo_19965230" type="hidden" value="생활가전_청소기"/>
 <input id="min_price_19965230" type="hidden" value="890820"/>
 <div class="prod_main_info">
 <div class="thumb_image">
 <a class="thumb_link click_log_product_standard_img_" href="https://prod.danawa.com/info/?pcode=19965230&amp;keyword=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&amp;cate=10325109" id="thumbLink_19965230" onmousedown="_trkEventLog('통합검색_상품블로그_유입');" target="_blank">
 <img alt="코드제로 A9S AX9884" fetchpriority="high" onerror="this.onerror=null; this.src='//img.danawa.com/new/noData/img/noImg_160.gif';" src="//img.danawa.com/prod_img/500000/230/965/img/19965230_1.jpg?shrink=130:130&amp;_v=20240311100128"/>
 </a>
 <a class="btn_view_zoom" data-assemblygallerycategory="N" href="#" id="imageZoom_19965230" onclick="return false;">
 <span class="ico i_zoom_vw"></span>이미지보기
 										</a>
 <input id="hidden_cate_sub_c1" type="hidden" value="72"/>
 

In [6]:
prodItems = soup.select('ul.product_list > li.prod_item')

In [7]:
# id="productItem19965230" 식으로 전개되므로 productItem이 아니면 패스
# 제품명 가져오기, 만약 애드스마트 영역이면 예외발생
prodItems[0].select('p.prod_name > a')[0].text.split()

['LG전자', '오브제컬렉션', '코드제로', 'A9S', 'AX9884']

In [8]:
# 제품스펙 가져오기
prodItems[0].select('div.spec_list')[0].text.strip().replace('\t', '')

'핸디스틱청소기 / 무선 / 흡입+물걸레(동시) / 흡입력: 250W / 소비전력: 620W / 2023년형 / [구성] 거치대: 올인원타워 / 먼지비움 / 충전 / UVC LED / 액서서리수납 / 스탠드거치 / 먼지비움시간: 30초 / 브러쉬: 와이드바닥 / 물걸레: 스팀, 고온, 일반 / 솔형 / 틈새 / 먼지봉투: 2.5L / [배터리] 사용시간: 30분(최대) / 충전시간: 4시간 / 분리형(2개) / 리튬이온 / 25.2V / [청소] 싸이클론흡입 / LED라이트 / 자동물공급 / 스마트인버터모터 / [부가] 스마트폰제어 / 대화형알림창 / 워셔블헤파필터 / 5단계여과 / 색상: 카밍베이지 / 무게: 2.47kg / 크기(가로x세로x깊이): 300x1120x245mm'

In [9]:
# 제품가격 가져오기
# 책에서는 1위 데이터를 가져오도록 했음
# 우리는 input 중 두번째 값이 최소값을 담고 있는 것을 확인
prodItems[0].select('input')[1].get('value')

'890820'

In [10]:
# 제품 인트로
prodItems[0].select('p.intro_text')[0].text

'가장 프리미엄 라인업(AX)으로 스팀물걸레 포함 구성, AX9988보다 흡입력만 낮음'

#### 1번 페이지에서 상품정보 추출

In [11]:
prodData = []

for prodItem in prodItems:
    try:
        prodName = prodItem.select('p.prod_name > a')[0].text.strip()   # 상품명 가져오기
        specList = prodItem.select('div.spec_list')[0].text.strip().replace('\t', '')   # 상품 스팩목록 가져오기
        price = prodItem.select('input')[1].get('value') # 최저가 가져오기
        prodData.append([prodName, specList, price])
    except:
        pass

In [12]:
# 결과확인
prodData

[['LG전자 오브제컬렉션 코드제로 A9S AX9884',
  '핸디스틱청소기 / 무선 / 흡입+물걸레(동시) / 흡입력: 250W / 소비전력: 620W / 2023년형 / [구성] 거치대: 올인원타워 / 먼지비움 / 충전 / UVC LED / 액서서리수납 / 스탠드거치 / 먼지비움시간: 30초 / 브러쉬: 와이드바닥 / 물걸레: 스팀, 고온, 일반 / 솔형 / 틈새 / 먼지봉투: 2.5L / [배터리] 사용시간: 30분(최대) / 충전시간: 4시간 / 분리형(2개) / 리튬이온 / 25.2V / [청소] 싸이클론흡입 / LED라이트 / 자동물공급 / 스마트인버터모터 / [부가] 스마트폰제어 / 대화형알림창 / 워셔블헤파필터 / 5단계여과 / 색상: 카밍베이지 / 무게: 2.47kg / 크기(가로x세로x깊이): 300x1120x245mm',
  '890820'],
 ['삼성전자 비스포크 제트 VS20B956AX',
  '핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 220W / 2022년형 / [구성] 거치대: 청정스테이션 / 먼지비움 / 충전 / UVC LED / 스탠드거치 / 먼지비움시간: 14초 / 브러쉬: 바닥 / 물걸레: 별매 / 솔형 / 틈새 / 연장관 / 먼지봉투: 1.2L / [배터리] 사용시간: 1시간(최대) / 충전시간: 3시간30분 / 분리형(1개) / 리튬이온 / 2200mAh / 25.2V / [청소] 싸이클론흡입 / LED라이트 / 디지털인버터모터 / 먼지통: 0.5L / [부가] 스마트폰제어 / 대화형알림창 / 디스플레이표시 / 배터리잔량표시 / 항균물통 / 워셔블헤파필터 / 5단계여과 / 색상: 페블그레이 / 크기(가로x세로x깊이): 250x930x202mm',
  '445230'],
 ['샤오미 미홀 M22',
  '핸디스틱청소기 / 무선 / 흡입형 / 소비전력: 320W / 2023년형 / [구성] 벽걸이거치 / 브러쉬: 바닥 / 물걸레: 별매 / 침구 / 솔형 / 틈새 / [배터리] 충전시간: 5시간 / 분리형(1개) / 2

##### 여러페이지 크롤링 하는 방법
- URL
    - https://search.danawa.com/dsearch.php?query=무선청소기&originalQuery=무선청소기&previousKeyword=무선청소기&checkedInfo=N&volumeType=allvs&page=1&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A

In [13]:
## 검색어, 페이지를 변경하면서 URL생성함수
def getSearchPageUrl(keyword, page):
    ecKeyword = parse.quote(keyword)
    url = f'https://search.danawa.com/dsearch.php?query={ecKeyword}&originalQuery={ecKeyword}&previousKeyword={ecKeyword}&checkedInfo=N&volumeType=allvs&' + \
          f'page={page}&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&' + \
            'recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&' + \
            'quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'
    return url

In [14]:
## 상품정보 추출하는 함수
def getProdItems(prodItems):    
    prodData = []

    for prodItem in prodItems:
        try:
            prodName = prodItem.select('p.prod_name > a')[0].text.strip()   # 상품명 가져오기
            specList = prodItem.select('div.spec_list')[0].text.strip().replace('\t', '')   # 상품 스팩목록 가져오기
            price = prodItem.select('input')[1].get('value') # 최저가 가져오기
            prodData.append([prodName, specList, price])
        except:
            pass
    
    return prodData

In [15]:
# 검색어에 ' '가 있으면 URL에서 링크 클릭이 안되는 문제 발생
getSearchPageUrl('마이크로 sd카드', 5)

'https://search.danawa.com/dsearch.php?query=%EB%A7%88%EC%9D%B4%ED%81%AC%EB%A1%9C%20sd%EC%B9%B4%EB%93%9C&originalQuery=%EB%A7%88%EC%9D%B4%ED%81%AC%EB%A1%9C%20sd%EC%B9%B4%EB%93%9C&previousKeyword=%EB%A7%88%EC%9D%B4%ED%81%AC%EB%A1%9C%20sd%EC%B9%B4%EB%93%9C&checkedInfo=N&volumeType=allvs&page=5&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'

In [16]:
total_page = 10

for page in tqdm(range(1, total_page+1)):
    # print('페이지 처리')    # tqdm 진행바가 여러번 출력
    time.sleep(1)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:10<00:00,  1.00s/it]


In [17]:
## 여러페이지 검색 후 크롤링하는 작업
driver = webdriver.Chrome()
# 암묵적으로 웹 자원 로드를 위해 3초정도 대기
driver.implicitly_wait(3.0)

keyword = '무선청소기'
total_page = 20
prodDataTotal = []  # 최종적으로 저장할 리스트

for page in tqdm(range(11, total_page+1)):
    # 검색 페이지 이동
    url = getSearchPageUrl(keyword, page)
    driver.get(url)
    # 페이지 로딩이 완료될때까지 5초간 대기
    time.sleep(5)
    
    # 현재 페이지 html 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 상품정보 추출
    prodItems = soup.select('ul.product_list > li.prod_item')
    prodItemList = getProdItems(prodItems)  # 리스트로 추출하는 함수
    
    # 추출된 정보를 prodDataTotal 추가
    prodDataTotal += prodItemList

100%|██████████| 10/10 [02:14<00:00, 13.45s/it]


In [18]:
dfprodData = pd.DataFrame(prodDataTotal)

In [19]:
dfprodData.columns = ['제품명', '제품스팩', '최저가']

In [20]:
dfprodData.to_excel('./data/다나와_무선청소기_결과2.xlsx', index=False)